# Testando pyarrow e seu desempenho em relação ao python com JSON, um case iremos lidar com strings de json em uma lista, no outro case sera uma lista de dicionarios normal, em ambas, será criado um df em pyspark, pyarrow demonstra superioridade

In [4]:
#!pip install pyspark
#!pip install pyarrow

In [1]:
import io, json
import pyarrow as pa
import pyarrow.json as paj
import pyarrow.parquet as pq
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("colab_spark_context").getOrCreate()

# lista de dicionarios python

In [6]:
data = []
for i in range(4000000):
    data.append({"nome": i+1, "idade": i+1})
data[0:2]


[{'nome': 1, 'idade': 1}, {'nome': 2, 'idade': 2}]

# transformando a lista de dicionarios no formato pyarrow table, levamos para parquet, pois ao que parece o pyarrow e o spark não se comunicam muito bem, mas ambos se comunicam bem com o parquet, por isso escrevemos um arquivo para acessar o df

In [7]:
x = pa.Table.from_pylist(data)
parquet_writer = pq.ParquetWriter('output_file.parquet', x.schema)
parquet_writer.write_table(x)
parquet_writer.close()
df = spark.read.parquet('output_file.parquet')

#ultimo teste: 8 segundos

In [8]:
df = spark.createDataFrame(data)
# mais de 60 segundos

### Acima vemos uma gritante diferença, mas ainda não parece usual, visto que dificilmente vemos objetos python sendo fonte (talvez para consumir API's pode ser uma boa opção). mas uma aplicação seria interessante no caso abaixo, no qual esses dicionarios python fossem uma string, tive de lidar com arquivos parquet com string json por uma medida paliativa por conta de configurações não feitas no synapse, dessa forma com certeza se tivesse um contato anterior com arrow nesse caso eu teria sido beneficiado, veja abaixo:

In [2]:
data2 = []
for i in range(4000000):
    data2.append(f'{{"nome": "Nome {i}", "idade": {i}}}')
data2[0:2]


['{"nome": "Nome 0", "idade": 0}', '{"nome": "Nome 1", "idade": 1}']

# Transformando as strings em objetos json do pyarrow, que escreve o parquet, acessando via dataframe spark

In [3]:


# o pyarrow permite que eu transforme essas strings em objetos como arquivo (file-like objects), que são acessíveis na memória, onde o pyarrow nos ajuda.
f = io.BytesIO(("\n".join(data2)).encode('utf-8'))

# com esses objetos conseguimos fazer a conversão para o formato json do pyarrow, o mesmo não tem compatibilidade com spark, então para acessarmos, escrevemos um arquivo parquet e lemos posteriormente
table = paj.read_json(f)

#escrevendo arquivo
parquet_writer = pq.ParquetWriter('output_file.parquet', table.schema)

#escrevendo a tabela
parquet_writer.write_table(table)

parquet_writer.close()
df = spark.read.parquet('output_file.parquet')

#ultimo teste: 10 segundos

# Voltando ao python, a opção que temos é ler individualmente cada string e parsea-la, o que acaba sendo muito desvantajoso em relação ao desempenho, visto que não há a possibilidade de acessar os objetos da memória.

In [5]:
table2 = [json.loads(y) for y in data2]
df2 = spark.createDataFrame(table2)
# ultimo teste: quase dois minutos